In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import itertools
import os
import urllib
import tqdm

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import urllib
import requests
from PIL import Image
from io import BytesIO
import logging

import pandas as pd

XPATH = "xpath"
erwo = "*error"
dash = '-'
slash = '/'
logger = logging.getLogger(__name__)

In [ ]:
# Set general settings
url_main = "https://decklist.tistory.com/"
xp_title = '//*[@id="container"]/main/div/div[2]/div[1]/div/div/h2'
xp_man = '//*[@id="container"]/main/div/div[2]/div[2]/div[2]/table/tbody/tr[1]/td[2]'

# Create headless ChromeOptions object
options = Options()
options.add_argument('--headless')

# Create a new ChromeDriverService object with the path to the Chromedriver executable
service = Service('C://chromedriver.exe')

# Initialize Chrome driver with headless options
driver = webdriver.Chrome(service=service, options=options)


In [ ]:
def xp_deck_ind(ind):
    return f'//*[@id="container"]/main/div/div[2]/div[2]/div[2]/p[{ind}]'

In [ ]:
def get_deck_info(driver, url_numb):
    url_comb = url_main + str(url_numb)
    driver.get(url_comb)

    # Get title of the Game
    try:
        title_element = driver.find_element(By.XPATH, xp_title).text
        game_cs = title_element.split("(")[0].strip()
        game_date = title_element.split(")")[1].strip()


    except:
        title_element = erwo
        game_cs = erwo
        game_date = erwo
        logger.error(f'{url_numb} | Failed to get | Title info')    

    

    # Get Number of Players
    try:
        man_element = driver.find_element(By.XPATH, xp_man).text
    except:
        man_element = erwo
        logger.error(f'{url_numb} | Failed to get | Man info')  

    # Get Name of the Decks
    deck_element_scrap = []
    i = 1
    while True:
        try:
            element = driver.find_element(By.XPATH, xp_deck_ind(i))
            deck_element_scrap.append(element.text)
            i += 1
        except NoSuchElementException:
            break

    for i in range(len(deck_element_scrap)):
        while '\n' in deck_element_scrap[i]:
            deck_element_scrap[i] = deck_element_scrap[i].replace('\n', '')

    # Modify Name of the Decks
    deck_elements = (filter(lambda x: x != '', deck_element_scrap))
    deck_elements = list(filter(lambda x: x != ' ', deck_elements))
    # # deck_elements = [s.replace('\n', '') for s in deck_elements]
    # for i in range(len(deck_elements)):
    #     while '\n' in deck_elements[i]:
    #         deck_elements[i] = deck_elements[i].replace('\n', '')


    deck_elements_dash = []
    for deck in deck_elements:
        if dash in deck:
            deck = deck.split(dash)[1]
        deck_elements_dash.append(deck)

    deck_elements_slash = []
    for deck in deck_elements_dash:
        if slash in deck:
            deck = deck.split(slash)
            for d in deck:
                deck_elements_slash.append(d)
        else:
            deck_elements_slash.append(deck)

    deck_elements = [s.strip() for s in deck_elements_slash]
    deck_elemetns_text = ' '.join(deck_element_scrap)

    # Get the Images
    img_elements = driver.find_elements(By.TAG_NAME, 'img')
    db_comb = []
    for i, img_element in enumerate(img_elements):
        deck_code = f"{url_numb:04d}-{i+1:03d}"
        filename = f"C:\ML_YGO\Deck_Log\{deck_code}.jpg"
        img_url = img_element.get_attribute('src')

        # Skip GIF images
        if img_url.startswith('data:image/gif'):
            continue  

        # Get the size of the images
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))
        width, height = img.size

        # Skip small images
        if (width == 343 and height == 353) or width < 50:
            pass
        elif width > 0:
            # Download Image
            urllib.request.urlretrieve(img_url, filename)

            if len(deck_elements) <= i:
                deck_element = erwo
            else:
                deck_element = deck_elements[i]
            
            db_join = [  
                deck_code,
                game_cs,
                game_date,
                man_element, 
                deck_element,
                title_element, 
                deck_elemetns_text
            ]
            db_comb.append(db_join)
    return db_comb

In [ ]:
#5~594
#600~1003
#1004~1207
#1317~1334
#1334~1347
#600~812 812~1003

In [ ]:
master_comb =[]
for i, url_numb in enumerate(range(800,812)):
    print(f'{url_numb} | Running')
    try: 
        db_comb = get_deck_info(driver,url_numb)
    except:
        print(f'{url_numb} | Unexpected Error')
        break
    master_comb.append(db_comb)
    if i% 5 == 0:
        time.sleep(2)
        print("-- Waiting to cool down --")

master_comb_clean = []
for first in master_comb:
    for second in first:
        joined_str = "\t".join(second)
        master_comb_clean.append (joined_str)

dir_write = 'C:\ML_YGO\Output\deck_stack.txt'
open(dir_write, 'w').close()
with open(dir_write , 'w', encoding="utf-8") as fp:
    for item in master_comb_clean:
        fp.write("%s\n" % item)
os.startfile(dir_write)


In [ ]:
url_numb = 400
url_comb = url_main + str(url_numb)
driver.get(url_comb)

# Get Name of the Decks
deck_element_scrap = []
i = 1
while True:
    try:
        element = driver.find_element(By.XPATH, xp_deck_ind(i))
        deck_element_scrap.append(element.text)
        i += 1
    except NoSuchElementException:
        break



# Modify Name of the Decks
deck_elements = (filter(lambda x: x != '', deck_element_scrap))
deck_elements = list(filter(lambda x: x != ' ', deck_elements))

# deck_elements = [s.replace('\n', '') for s in deck_elements]

for i in range(len(deck_elements)):
    while '\n' in deck_elements[i]:
        deck_elements[i] = deck_elements[i].replace('\n', '')



In [ ]:
print(deck_element_scrap)

print(deck_elements)

# Old Examples

In [ ]:
title_element = driver.find_element(By.XPATH, xp_title)
man_element = driver.find_element(By.XPATH, xp_man)
img_elements = driver.find_elements(By.TAG_NAME, 'img')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import urllib

XPATH = "xpath"
TAG_NAME = "tag_name"

# Create headless ChromeOptions object
options = Options()
options.add_argument('--headless')

# Initialize Chrome driver with headless options
driver = webdriver.Chrome('C://cromedriver.exe', options=options)

url_main = "https://decklist.tistory.com/"

for i in range(1010, 1208):
    url_numb = i
    url_comb = url_main + str(url_numb)
    driver.get(url_comb)

    img_elements = driver.find_elements(By.TAG_NAME, 'img')

    for i in range(len(img_elements)):
        deck_code = f"{url_numb:04d}-{i+1:03d}"
        filename = f"C:\ML_YGO\Deck_Log\{deck_code}.jpg"
        img_element = img_elements[i]
        img_url = img_element.get_attribute('src')
        urllib.request.urlretrieve(img_url, filename)
    
    time.sleep(0.5)

# Close the driver
driver.quit()
